In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
import warnings
warnings.filterwarnings('ignore')
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
#preprocessing of CSV 
data = []
file_list = ['groups_0.xlsx','groups_04.csv','groups_05.csv']
#file_list = ['groups_05.csv']
for file_path in file_list[0]:
    file1 = df=pd.read_excel('group_0.xlsx')
    t = file1.drop_duplicates(['meetup_0_description'], keep='last')
    text_list = t['meetup_0_description'].tolist()
    text_list = list(map(str, text_list))
    for text in text_list:
        data.append(text)

In [9]:
meetup_words = ['<p>','<\p>','<br>','http','idea','meeting','need','organization','member','club','group', 'join','meetup','right','idea','join','community','make','social','discuss','local','chapter',
                    'people','country','br','www','span','like','event','nyyrc','www','facebook','people','href','com','facebook','topic','human','justice','discus','important','_','n•','meet','welcome','event','get','bring','day','join','go','https',
                'nbsp','medium','ideas','one','-','style','party','font-size','color','email','small','youtube','others','org','state','level']
from nltk.corpus import stopwords
states = ['alabama', 'alaska', 'austrian','arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida',
            'georgia', 'hawaii', 'idaho', 'illinois', 'indiana'
            , 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota',
            'mississippi', 'missouri','montana', 'nebraska', 'nevada', 'new', 'hampshire', 'new', 'jersey', 'new', 'mexico', 'new', 'york', 'nyc',
            'north carolina', 'north dakota', 'ohio',
            'oklahoma', 'oregon', 'pennsylvania', 'rhode', 'island', 'south', 'carolina', 'south', 'dakota', 'tennessee',
            'texas', 'utah', 'vermont', 'virginia',
            'washington', 'dc', 'west', 'Virginia', 'wisconsin', 'wyoming', 'philadelphia', 'brooklyn', 'america', 'pittsburgh']

stop_words = stopwords.words('english')
stop_words.extend(meetup_words)
stop_words.extend(states)

In [10]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 7

vectorizer = CountVectorizer(
                             stop_words=stop_words, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])

 

(2470, 7)
[ 0.00170346  0.00170436  0.00170231  0.00170315  0.98978068  0.00170248
  0.00170355]


In [11]:
def print_topics(model, vectorizer, top_n=20):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('volunteer', 102.93438047987473), ('life', 89.587013457316203), ('share', 80.752992423120233), ('help', 68.263215160826064), ('freedom', 65.349654005129395), ('want', 64.839498478396209), ('liberty', 62.403049776220477), ('world', 62.305461508141398), ('civic', 61.952174679043395), ('jewish', 61.839657534876196), ('friends', 58.82956431158928), ('gather', 55.850842340090878), ('something', 52.969894627871149), ('issues', 52.698865493128942), ('standing', 52.67999682032999), ('know', 52.637186001853443), ('good', 50.286701836992322), ('adr', 49.969785368934161), ('around', 49.887072434264155), ('plan', 49.838036716899964)]
Topic 1:
[('christian', 366.44992915386774), ('support', 227.84729292863062), ('national', 222.89510954378369), ('let', 163.75057839406307), ('please', 154.53611368203983), ('constitution', 148.75319597116805), ('oath', 147.98031150839503), ('government', 137.7788372699932), ('north', 127.24758764039413), ('federal', 102.7235855929038), ('events'

In [12]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model)
vis

TypeError: prepare() missing 2 required positional arguments: 'corpus' and 'dictionary'